<a href="https://colab.research.google.com/github/SanjaySaatyaki/fast_ai_try_out/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 79.0 MB/s eta 0:00:00
Mounted at /content/gdrive


In [2]:

#hide
from fastbook import *
from IPython.display import display,HTML

In [3]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [4]:
!ls {path}

imdb.vocab  README  test  tmp_clas  tmp_lm  train  unsup


In [5]:
files = get_text_files(path, folders=['train','test','unsup'])

In [6]:
txt = files[0].open().read(); txt[:75]

'This film was full of suspense and was well directed, the black and white e'

In [7]:
spacy = WordTokenizer()
toks = first(spacy([txt]))

In [8]:
print(coll_repr(toks, 30))

(#188) ['This','film','was','full','of','suspense','and','was','well','directed',',','the','black','and','white','effect','made','it','a','great','mystery','.','Fay','Emerson,(Hilda','Fenchurch',')','who','was','married','twice'...]


In [9]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt),31))

(#217) ['xxbos','xxmaj','this','film','was','full','of','suspense','and','was','well','directed',',','the','black','and','white','effect','made','it','a','great','mystery','.','xxmaj','fay','emerson,(hilda','xxmaj','fenchurch',')','who'...]


In [10]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [11]:
txts = L(o.open().read() for o in files[:2000])

In [12]:
def subword(sz):
  sp = SubwordTokenizer(vocab_sz=sz)
  sp.setup(txts)
  return ' '.join(first(sp([txt])))[:400]

In [13]:
subword(10000)

"▁This ▁film ▁was ▁full ▁of ▁suspense ▁and ▁was ▁well ▁directed , ▁the ▁black ▁and ▁white ▁effect ▁made ▁it ▁a ▁great ▁mystery . ▁Fay ▁Emerson , ( H il da ▁F en church ) ▁who ▁was ▁married ▁twice ▁to ▁the ▁famous ▁musician ▁S k itch ▁He n der son ▁and ▁also ▁the ▁son ▁of ▁Elliot t ▁Roosevelt , ▁( F D R ' s ▁Son ) ▁fell ▁mad ly ▁in ▁love ▁with ▁Zac har y ▁Scott ( ▁Ro nnie ▁Ma son / Mar sh ). ▁Ro nni"

In [14]:

toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#217) ['xxbos','xxmaj','this','film','was','full','of','suspense','and','was','well','directed',',','the','black','and','white','effect','made','it','a','great','mystery','.','xxmaj','fay','emerson,(hilda','xxmaj','fenchurch',')','who'...]


In [15]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#217) ['xxbos','xxmaj','this','film','was','full','of','suspense','and','was','well','directed',',','the','black','and','white','effect','made','it'...]

In [16]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#2040) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','and','a','of','to','is','i','in','it'...]"

In [17]:

nums = num(toks)[:20]; nums

TensorText([  2,   8,  20,  31,  26, 309,  14, 926,  12,  26,  76, 611,  11,   9, 310,  12, 392, 612, 119,  19])

In [18]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj this film was full of suspense and was well directed , the black and white effect made it'

In [19]:

#hide_input
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [20]:
nums200 = toks200.map(num)

In [21]:
dl = LMDataLoader(nums200)

In [22]:
x,y = first(dl)
x.shape, y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [23]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj this film was full of suspense and was well directed , the black and white effect made it'

In [24]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj this film was full of suspense and was well directed , the black and white effect made it a'

In [25]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [26]:
dls_lm.show_batch(max_n=2)

xxbos xxmaj oscar xxmaj wilde 's comedy of manners , perhaps the wittiest play ever written , is all but wrecked at the hands of a second - rate cast . xxmaj sanders is , as one would expect , casually , xxunk brilliant in the role of xxmaj lord xxmaj xxunk , but the rest of the cast makes the entire procedure a waste of time . xxmaj jean xxmaj crain attempts a stage accent in alternate sentences and
! xxbos xxmaj scary xxmaj movie 2 was a grave disappointment . xxmaj simply referencing movies , like xxmaj mission xxmaj impossible 2 does not lead to comedy . xxmaj the movie opens well enough with a funny white people rapping scene and an excellent use of xxmaj james xxmaj woods ands xxmaj andy xxmaj richter . xxmaj the movie plummets from there unfortunately . xxmaj the acting is awful . xxmaj tori xxmaj spelling looks and acts completely out
xxmaj oscar xxmaj wilde 's comedy of manners , perhaps the wittiest play ever written , is all but wrecked at the hands of a second - rate cast .

In [28]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
learn.fit_one_cycle(1, 2e-2)

/usr/local/lib/python3.12/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.12/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,perplexity,time


In [ ]:
learn.save('1epoch')

In [ ]:
learn = learn.load('1epoch')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

In [ ]:
learn.save_encoder('finetuned')

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75)
         for _ in range(N_SENTENCES)]

In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)


In [ ]:

learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,
                                metrics=accuracy).to_fp16()

In [ ]:
learn = learn.load_encoder('finetuned')

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))